In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 125
import seaborn as sns
from Bio import SeqIO, Seq
import scipy.stats as st
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import glob, os, yaml, subprocess, itertools, sparse, pickle

who_variants = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv")
pd.set_option('display.float_format', lambda x: '%.4f' % x)
samples_summary = pd.read_csv("../data/samples_summary.csv")
analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'

In [2]:
drug = "Delamanid"
# df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv"))

# df_genos = pd.read_csv(os.path.join(analysis_dir, drug, "genos.csv.gz"), compression="gzip")
# df_genos["orig_variant"] = df_genos["resolved_symbol"] + "_" + df_genos["variant_category"]

In [100]:
# final_analysis file with all significant variants for a drug
res_df = pd.read_csv(os.path.join(analysis_dir, drug, "final_analysis.csv"))

In [101]:
res_df

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,confidence_WHO_2021,Odds_Ratio,OR_LB,...,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,predicted_effect
0,ddn_lof,0.2951,0.1698,0.3745,0.0000,0.0000,0.0000,NaN,1.3432,1.1851,...,0.9500,0.0446,0.1092,0.9991,0.9999,73.3231,852.3245,0.8908,0.9559,lof
1,fgd1_lof,0.1563,-0.0000,0.2080,0.0041,0.0332,0.2990,NaN,1.1692,1.0000,...,1.0000,0.0000,0.0194,1.0000,1.0000,inf,inf,0.9806,1.0000,lof
2,ddn_p.Val127Phe,0.1547,-0.0273,0.2292,0.0025,0.0299,0.1792,NaN,1.1673,0.9731,...,1.0000,0.0000,0.0263,0.9996,1.0000,0.0000,inf,0.9739,1.0002,missense_variant
3,ddn_p.Leu49Pro,0.1377,0.1170,0.1530,0.0000,0.0000,0.0000,2) Assoc w R - Interim,1.1476,1.1241,...,1.0000,0.0210,0.0723,0.9996,1.0000,75.1657,inf,0.9278,0.9792,missense_variant
4,fbiB_p.Lys448Arg,0.1318,-0.0342,0.2785,0.0537,0.1189,1.0000,3) Uncertain significance,1.1409,0.9664,...,0.1667,0.0410,0.1087,0.9853,0.9895,3.2246,9.1374,0.9028,0.9713,missense_variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,fbiB_p.Leu447Arg,-0.0649,-0.0788,-0.0504,0.0000,0.0000,0.0000,3) Uncertain significance,0.9371,0.9242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missense_variant
954,fgd1_c.465C>T,-0.0783,-0.0901,-0.0657,0.0000,0.0000,0.0000,NaN,0.9247,0.9139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,synonymous_variant
955,fbiA_p.Arg304Gln,-0.0799,-0.1325,0.0000,0.0480,0.1129,1.0000,3) Uncertain significance,0.9232,0.8759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missense_variant
956,ndh_lof,-0.1093,-0.1496,-0.0602,0.0000,0.0000,0.0001,NaN,0.8964,0.8610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lof


In [102]:
res_df.columns

Index(['orig_variant', 'coef', 'coef_LB', 'coef_UB', 'pval', 'BH_pval',
       'Bonferroni_pval', 'confidence_WHO_2021', 'Odds_Ratio', 'OR_LB',
       'OR_UB', 'Tier', 'Phenos', 'unpooled', 'Num_Isolates', 'Total_Isolates',
       'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'LR+', 'LR-', 'PPV_LB',
       'PPV_UB', 'Sens_LB', 'Sens_UB', 'Spec_LB', 'Spec_UB', 'LR+_LB',
       'LR+_UB', 'LR-_LB', 'LR-_UB', 'predicted_effect'],
      dtype='object')

In [103]:
res_df.loc[pd.isnull(res_df["Sens"])]

,orig_variant,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,confidence_WHO_2021,Odds_Ratio,OR_LB,...,PPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB,predicted_effect
11,PC4,0.1016,-0.0728,0.2469,0.1077,0.1228,1.0000,NaN,1.1069,0.9298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,PC2,0.0605,-0.1190,0.2391,0.2571,0.2844,1.0000,NaN,1.0624,0.8878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,PC3,0.0300,-0.1110,0.1728,0.3431,0.3738,1.0000,NaN,1.0305,0.8950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,PC1,-0.0039,-0.1845,0.1639,0.4829,0.5109,1.0000,NaN,0.9961,0.8316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
957,PC0,-0.2291,-0.4017,-0.0775,0.0026,0.0274,0.1919,NaN,0.7953,0.6692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Combined Analysis Files

In [ ]:
def final_processing(drug):
    '''
    Functions for processing outputs before sending to everyone else.
    
    1. Remove principal components (will describe them separately)
    2. Add LOF to the predicted_effect column for pooled LOF mutations
    3. Remove genome_index column (should actually do that earlier, but will fix later)
    4. Remove the logistic regression coefficient columns (they will prefer to work with odds ratios)
    5. Any other column renaming or dropping for clarity
    '''
    
    analysis_df = pd.read_csv(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/{drug}/final_analysis.csv")
    analysis_df.rename(columns={"orig_variant": "mutation", "Tier1_only": "Tier", "WHO_phenos": "Phenos"}, inplace=True)
    
    # remove logReg coefficients. Keep only odds ratios. Remove the other two columns, which were present mainly for me to see
    # if we were picking up many mutations that were in the 2021 mutation catalog
    del analysis_df["genome_index"]
    del analysis_df["confidence_WHO_2021"]
    analysis_df = analysis_df[analysis_df.columns[~analysis_df.columns.str.contains("coef")]]
    
    # remove significant principal components and replace the NaNs in the predicted effect column for the gene loss of functions
    analysis_df = analysis_df.loc[~analysis_df["mutation"].str.contains("PC")]
    analysis_df.loc[analysis_df["mutation"].str.contains("lof"), "predicted_effect"] = "LOF"
    
    # predicted effect should not be NaN for anything. position is NaN only for the pooled LOF mutations
    assert len(analysis_df.loc[pd.isnull(analysis_df["predicted_effect"])]) == 0
    assert len(analysis_df.loc[(~analysis_df["mutation"].str.contains("lof")) & (pd.isnull(analysis_df["position"]))]) == 0
    
    analysis_df["Tier"] = analysis_df["Tier"].map({1: 1, 0: 2})
    analysis_df["Phenos"] = analysis_df["Phenos"].map({1: "WHO", 0: "ALL"})
    
    # reorder columns
    analysis_df = analysis_df[['mutation', 'predicted_effect', 'position', 'Odds_Ratio', 'OR_LB', 'OR_UB', 'pval', 'BH_pval', 'Bonferroni_pval',
       'Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'Sens_LB', 'Sens', 'Sens_UB', 'Spec_LB', 'Spec', 'Spec_UB', 'PPV', 'PPV_LB', 'PPV_UB',
       'LR+_LB', 'LR+', 'LR+_UB', 'LR-_LB', 'LR-', 'LR-_UB', 'Tier', 'Phenos', 'poolLOF', 'Syn']]
    
    analysis_df[['poolLOF', "Syn"]] = analysis_df[['poolLOF', "Syn"]].astype(int)
        
    return analysis_df

In [ ]:
analysis_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis"
finished_drugs = []

for drug in os.listdir(analysis_dir):
    
    if os.path.isfile(os.path.join(analysis_dir, drug, "final_analysis.csv")):
        finished_drugs.append(drug)
        # drug_analyses.append(final_processing(drug))

In [ ]:
print(finished_drugs)

In [ ]:
# those that are actually done
finished_drugs = ['Pyrazinamide', 'Amikacin', 'Clofazimine', 'Linezolid', 'Moxifloxacin', 'Kanamycin', 'Bedaquiline', 'Capreomycin', 'Delamanid']

drug_analyses = {}
for drug in finished_drugs:
    drug_analyses[drug] = final_processing(drug)
    
# write results to an Excel file, where each sheet is named for a drug
with pd.ExcelWriter("Farhat_logReg_analysis.xlsx") as file:
   
    for key, val in drug_analyses.items():
        val.to_excel(file, sheet_name=key, index=False)

In [ ]:
drug_analyses[4].query("Tier==2&Phenos=='WHO'")

In [ ]:
drug_analyses[0].query("Tier==1&Phenos=='ALL'")

In [ ]:
drug_analyses[0].query("predicted_effect=='synonymous_variant' & ")

In [ ]:
moxi_df.loc[pd.isnull(moxi_df["position"])]

# Catalog-Based Method Comparison

In [ ]:
def get_logReg_summary(out_dir):
    
    model_summary = pd.read_csv(os.path.join(out_dir, "logReg_summary.csv"))
    model_analysis = pd.read_csv(os.path.join(out_dir, "model_analysis.csv"))
    
    summary_sens = model_summary.loc[0, "Sens"]
    summary_spec = model_summary.loc[0, "Spec"]
    
    #max_idx = [np.argmax(model_analysis["Sens"]), np.argmax(model_analysis["Spec"])]
    
    better_variant = model_analysis.query("Sens >= @summary_sens & Spec >= @summary_spec")
    
    if len(better_variant) == 0:
        print("No variants have comparable sensitivity AND specificity")
    else:
        print(better_variant)
    return model_summary, model_analysis
    
    #return model_summary, model_analysis.iloc[max_idx, :][["orig_variant", "coef", "Sens", "Spec", "accuracy", "balanced_accuracy"]]

In [ ]:
#out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Levofloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn"

summary, analysis = get_logReg_summary(out_dir)
summary

In [ ]:
analysis.query("Sens > 0.9006")

In [ ]:
analysis.loc[analysis["orig_variant"].str.contains("PC")]

In [ ]:
analysis.query("Spec > 0.9575")

In [ ]:
out_dir = "/n/data1/hms/dbmi/farhat/ye12/who/analysis/Moxifloxacin/tiers=1+2/phenos=ALL/dropAF_withSyn"

get_logReg_summary(out_dir)

In [ ]:
df_pza = pd.read_csv("/n/data1/hms/dbmi/farhat/ye12/who/analysis/Pyrazinamide/tiers=1+2/phenos=WHO/dropAF_withSyn/phenos.csv")

# the missing ones might be M. cannettii, most similar to L6 based on the other lineage callers
lineages = pd.read_pickle("../data/combined_lineage_sample_IDs.pkl")
lineages["Lineage"] = lineages["Lineage"].fillna("6")
lineages["Lineage_1"] = lineages["Lineage_1"].fillna("6")

lineages = lineages[["Sample Name", "Sample ID", "Lineage_1"]]
lineages["Lineage"] = [str(val).split(".")[0] for val in lineages["Lineage_1"].values]
lineages.loc[lineages["Lineage"].str.contains("BOV"), "Lineage"] = "M. bovis"

assert len(lineages.loc[pd.isnull(lineages["Lineage"])]) == 0

########## KEEP ONLY ISOLATES WITH ALL 3 PIECES OF DATA ##########

# get only isolates with data for everyting: SNP matrix, in the model, and lineages
combined = lineages.merge(df_pza, left_on="Sample ID", right_on="sample_id")

In [ ]:
combined.groupby("Lineage")["phenotype"].mean().sort_values(ascending=False)

In [ ]:
combined.query("Lineage == 'M. bovis'").Lineage_1.unique()